In [230]:
plain_vowels = ['а', 'о', 'у', 'ы', 'э', 'и']
j_vowels = ['я', 'е', 'ё', 'ю']
non_front_vowels = ['а', 'о', 'у', 'ы', 'э']
front_vowels = ['я', 'е', 'ё', 'ю','и']
sonant_cons = ['л', 'м', 'р', 'н']
voiced_cons = ['б', 'в', 'г', 'д','з', 'ж']
unvoiced_cons = ['п', 'ф', 'к', 'т','с', 'ш', 'х', 'ц', 'щ', 'ч']

devoiced = {'б': 'п', 'в': 'ф', 'г': 'к',
 'д': 'т', 'з': 'с', 'ж': 'ш', 'л': 'л',
 'м': 'м', 'р': 'р', 'н': 'н', 'п': 'п',
 'ф': 'ф', 'к': 'к', 'т': 'т', 'с': 'с',
 'ш': 'ш', 'х': 'х', 'ц': 'ц', 'щ': 'щ',
 'ч': 'ч'}

soft = {'б': "б'", 'в': "в'", 'г': "г'",
 'д': "д'", 'з': "з'", 'ж': 'ж',
 'п': "п'", 'ф': "ф'", 'к': "к'",
 'т': "т'", 'с': "с'", 'ш': 'ш',
 'х': "х'", 'ц': 'ц','щ': 'щ', 'ч': 'ч',
 'л': "л'", 'м': "м'", 'р': "р'", 'н': "н'"}

vocalized = {'п': 'б', 'ф': 'в', 'к': 'г', 'т': 'д',
 'с': 'з', 'ш': 'ж', 'х': 'х', 'ц': 'ц', 'щ': 'щ',
 'ч': 'ч', 'б': 'б', 'в': 'в', 'г': 'г', 'д': 'д',
 'з': 'з', 'ж': 'ж', 'л': 'л', 'м': 'м', 'р': 'р', 'н': 'н'}

to_plain = {'я': 'а', 'е': 'э', 'ё': 'о',
 'ю': 'у', 'а': 'а', 'о': 'о', 'у': 'у',
 'ы': 'ы', 'э': 'э', 'и': 'и'}

In [255]:
def start(char, previous):
    if char in plain_vowels:
        return start, char, char
    if char in j_vowels:
        return start, 'j' + to_plain[char], char
    if char in voiced_cons:
        return voiced, '', char
    if char in unvoiced_cons:
        return unvoiced, '', char
    if char in sonant_cons:
        return sonant, '', char
    if char == 'й':
        return start, 'j', char
    if char == '$':
        return stop, '', char

def sonant(char, previous):
    if char in non_front_vowels:
        return start, previous + char, char
    if char in front_vowels:
        return start, soft[previous] + to_plain[char], char
    if char in voiced_cons:
        return voiced, previous, char
    if char in unvoiced_cons:
        return unvoiced, previous, char
    if char in sonant_cons:
        return sonant, previous, char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == 'ь':
        return soft_sign, '', previous
    if char == 'ъ':
        return hard_sign, '', previous
    if char == '$':
        return stop, previous, char

def voiced(char, previous):
    if char in non_front_vowels:
        return start, previous + char, char
    if char in front_vowels:
        return start, soft[previous] + to_plain[char], char
    if char in voiced_cons:
        return voiced, previous, char
    if char in unvoiced_cons:
        return unvoiced, devoiced[previous], char
    if char in sonant_cons:
        return sonant, previous, char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == 'ь':
        return soft_sign, '', previous
    if char == 'ъ':
        return hard_sign, '', previous
    if char == '$':
        return stop, devoiced[previous], char

def unvoiced(char, previous):
    if char in non_front_vowels:
        return start, previous + char, char
    if char in front_vowels:
        return start, soft[previous] + to_plain[char], char
    if char in voiced_cons and not char == 'в':
        return voiced, vocalized[previous], char
    if char in unvoiced_cons or char == 'в':
        return unvoiced, previous, char
    if char in sonant_cons:
        return sonant, previous, char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == 'ь':
        return soft_sign, '', previous
    if char == 'ъ':
        return hard_sign, '', previous
    if char == '$':
        return stop, previous, char

def soft_sign(char, previous):
    if char in non_front_vowels or char in front_vowels:
        return start, soft[previous] + 'j' + to_plain[char], char
    if char in voiced_cons and not char == 'в':
        return voiced, soft[vocalized[previous]], char
    if char in unvoiced_cons or char == 'в':
        return unvoiced, soft[devoiced[previous]], char
    if char in sonant_cons or char == 'в':
        return sonant, soft[previous], char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == '$':
        return stop, soft[devoiced[previous]], char
    pass

def hard_sign(char, previous):
    if char in non_front_vowels or char in front_vowels:
        return start, soft[previous] + 'j' + to_plain[char], char
    
    
def stop(char):
    pass

In [238]:
def transcribe(string):
    state = start
    previous = ''
    output = ''
    try:
        for symbol in string:
            state, answer, previous = state(symbol, previous)
            output += answer
    except TypeError:
        return False, string # bad input
    return True, output

In [233]:
def preprocessing(text):
    tokens = []
    for word in text.split():
        token = []
        for symbol in word:
            if symbol.isalpha():
                token.append(symbol)
        if len(token) > 0:
            tokens.append(''.join(token).lower() + '$')
    return tokens

In [257]:
def transcribe_text(text):
    tokens = preprocessing(text)
    transcribed = []
    bad_input = []
    for word in words:
        approve, transcription = transducer(word)
        if approve:
            transcribed.append(transcription)
        else:
            bad_input.append(transcription)
    phonematic_text = ' '.join(transcribed)
    return phonematic_text, bad_input

In [254]:
def phonematic_search(sequence, text):
    transcription, bad_inputs = transcribe_text(text)
    if len(bad_inputs):
        line = ', '.join(bad_inputs)
        line = f'Unable to transcribe the following:{line}'
        print(line)
    return sequence in transcription

In [264]:
with open('Балашовская правда корпус/1000.txt', encoding='utf-8') as file:
    contents = file.read()
    
sequence = "чэлов'эк"
print(phonematic_search(sequence, contents))
sequence = "мар'ина"
print(phonematic_search(sequence, contents))

True
False
